### Topic - Structured output
##### Ensure responses adhere to a JSON scheme

In [1]:
import os

from openai import OpenAI
from pydantic import BaseModel

In [2]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## 🧾 Structured Response

**Structured Response**: Guides the AI to return answers in a specific format, such as JSON.

- Helps make the output easier to parse and use in applications.
- Useful when integrating AI with other systems or when consistent formatting is needed.



#### Step 1: Define the response format in a Pydantic model

**Context:** Let's say you are creating a AI Agent that can help you to book and schedule or chage appointments.

In [3]:
class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

#### Step 2: Call the model

In [4]:
completion = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "Extract the event information."},
        {
            "role": "user",
            "content": "Alice and Bob are going to a science fair on Friday.",
        },
    ],
    response_format=CalendarEvent,
)

#### Step 3: Parse the response

In [5]:
event = completion.choices[0].message.parsed


In [6]:
event

CalendarEvent(name='Science Fair', date='2023-11-03', participants=['Alice', 'Bob'])

In [7]:
event.name


'Science Fair'

In [8]:
event.date


'2023-11-03'

In [9]:
event.participants

['Alice', 'Bob']

**Usecase:** Let's say we have a Google Calendar API and we can create an event. 

> 📖 More details of Structured Outputs on the [OpenAI Docs](https://platform.openai.com/docs/guides/structured-outputs?api-mode=responses)

## LATEST

#### Using a Pydantic model (and simple response format)

In [10]:
response = client.responses.parse(
    model="gpt-4o",
    input="Alice and Bob are going to a science fair on Friday",
    instructions="Extract the event information",
    text_format=CalendarEvent,
)

In [11]:
response_model = response.output[0].content[0].parsed

In [12]:
type(response_model)

__main__.CalendarEvent

In [13]:
response_model.name

'Science Fair'

In [14]:
response_model.date

'Friday'

In [15]:
response_model.participants

['Alice', 'Bob']

#### Real-world example


In [16]:
class ListingSearch(BaseModel):
    location: str
    intent: str
    price_range: str | None = None
    property_type: str | None = None
    bedrooms: int | None = None


In [24]:

completion = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=[
        {
            "role": "system",
            "content": "Extract structured real estate search parameters from the user's message.",
        },
        {
            "role": "user",
            "content": "Find me a 3-bedroom house in San Francisco under $1 million.",
        },
    ],
    response_format=ListingSearch,
)

In [25]:
search_params = completion.choices[0].message.parsed

In [26]:
search_params

ListingSearch(location='San Francisco', intent='buy', price_range='under $1 million', property_type='house', bedrooms=3)

In [27]:
import json
with open("listing.json", "r") as file:
    listings = json.load(file)

In [28]:
def filter_listings(listings, search_params):
    filtered = []
    for listing in listings:
        if (
            listing["location"].lower() == search_params.location.lower()
            and listing["intent"].lower() == search_params.intent.lower()
            and listing["property_type"].lower() == search_params.property_type.lower()
            and listing["bedrooms"] == search_params.bedrooms
            and (not search_params.price_range or listing["price"] <= 1000000)
        ):
            filtered.append(listing)
    return filtered

In [29]:
matching_listings = filter_listings(listings, search_params)

In [30]:
if matching_listings:
    print("Matching Listings:")
    for listing in matching_listings:
        print(
            f"- {listing['bedrooms']}BHK {listing['property_type']} in {listing['location']} for ${listing['price']}"
        )
else:
    print("No matching listings found.")

Matching Listings:
- 3BHK house in San Francisco for $950000
